# ACA ESTAN LAS BASES DE DATOS Y UNA MUESTRA DE QUE ESTAN BIEN

In [33]:
import numpy as np
import pandas as pd
import os

In [34]:
norespondieron = pd.read_excel("norespondieron.xlsx")
respondieron = pd.read_excel("respondieron.xlsx")
print(norespondieron.shape)
print(respondieron.shape)
print(norespondieron.columns.tolist())
print(respondieron.columns.tolist())

(2985, 23)
(12866, 26)
['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'REGION', 'ANO4', 'CH04', 'CH06', 'CH07', 'CH08', 'CH12', 'CH13', 'CH14', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'PP10A', 'PP03J', 'PP04A', 'P21', 'P47T', 'ITF', 'IPCF', 'adulto_equiv']
['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'REGION', 'ANO4', 'CH04', 'CH06', 'CH07', 'CH08', 'CH12', 'CH13', 'CH14', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'PP10A', 'PP03J', 'PP04A', 'P21', 'P47T', 'ITF', 'IPCF', 'adulto_equiv', 'ad_equiv_hogar', 'ingreso_necesario', 'pobre']


In [35]:
# Obtener los tipos de datos de cada df
tipos_no = norespondieron.dtypes.rename("norespondieron")
tipos_re = respondieron.dtypes.rename("respondieron")

# Unirlos en un solo DataFrame
comparacion_tipos = pd.concat([tipos_no, tipos_re], axis=1)

# Mostrar el cuadro resultante
print(comparacion_tipos)

                  norespondieron respondieron
CODUSU                    object       object
NRO_HOGAR                  int64        int64
COMPONENTE                 int64        int64
REGION                     int64        int64
ANO4                       int64        int64
CH04                       int64        int64
CH06                       int64        int64
CH07                       int64        int64
CH08                       int64        int64
CH12                       int64        int64
CH13                       int64        int64
CH14                       int64        int64
NIVEL_ED                   int64        int64
ESTADO                     int64        int64
CAT_INAC                   int64        int64
PP10A                      int64        int64
PP03J                      int64        int64
PP04A                      int64        int64
P21                        int64        int64
P47T                       int64        int64
ITF                        int64  

# A PARTIR DE ACA EMPEZAR A REALIZAR EL TP